In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%pip install geopy
%pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from transformers import pipeline
from geopy.distance import geodesic
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import torch

In [ ]:
# 데이터 로드
data = pd.read_csv("/content/drive/MyDrive/Project/hotels_with_coordinates.csv")
data

<ipython-input-4-05fb802ec9a8>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/content/drive/MyDrive/Project/hotels_with_coordinates.csv")


,hotel,star,review,date,price,address,district,length,clean_review,latitude,longitude
0,유성 메이호텔,5,매우 만족합니다 깨끗하고 시설도좋고 주변에 먹을것도많고 주차자리도 매우 널하고 1층...,2024.10.18,60000,대전 유성구 봉명동 551-17,유성구,250,매우 만족합니다 깨끗하고 시설도 좋고 주변에 먹을 것도 많고 주차 자리도 매우 널하...,36.354776,127.342764
1,유성 메이호텔,4,주변에 숙소가 많은데 다른숙소를 주로 이용하다 이번에 처음 이용해보았는데 객실도 너...,2024.10.18,60000,대전 유성구 봉명동 551-17,유성구,262,주변에 숙소가 많은데 다른 숙소를 주로 이용하다 이번에 처음 이용해 보았는데 객실도...,36.354776,127.342764
2,유성 메이호텔,5,봉명동 술집에서 술마실때 위치가 너무 좋아요 항상 찾는곳으로 주차장도 지상도 있고 ...,2024.10.28,60000,대전 유성구 봉명동 551-17,유성구,255,봉명동 술집에서 술 마실 때 위치가 너무 좋아요 항상 찾는 곳으로 주차장도 지상도 ...,36.354776,127.342764
3,유성 메이호텔,5,짐 놓고갔엇는데 프론트 분 이 되게 친절하셨었어요 아 송주차장이 없지만 그래도 주변...,2024.10.29,60000,대전 유성구 봉명동 551-17,유성구,192,짐 놓고 갔었는데 프런트 분 이 되게 친절하셨었어요 아 송 주차장이 없지만 그래도 ...,36.354776,127.342764
4,유성 메이호텔,4,할머니 댁같은 느낌객실깔끔한데 오래된 느낌은 못 지우겠어요 전담을 옆방에서 피는지 ...,2024.11.05,60000,대전 유성구 봉명동 551-17,유성구,259,할머니 댁 같은 느낌 객실 깔끔한데 오래된 느낌은 못 지우겠어요 전담을 옆방에서 피...,36.354776,127.342764
...,...,...,...,...,...,...,...,...,...,...,...
233552,대전 유성 발렌타인,3,안마방 싸고 젛음,2017.10.20,"20,000",대전광역시 유성구 장대로 20-19,유성구,9,안마방 싸고 젛음,36.357106,127.337561
233553,대전 유성 발렌타인,5,합리적인가격에 편히 깔끔하게있다갑니다,2017.10.20,"20,000",대전광역시 유성구 장대로 20-19,유성구,20,합리적인 가격에 편히 깔끔하게 있다 갑니다,36.357106,127.337561
233554,대전 유성 발렌타인,4,50원 할인되서 되게싸게 갔어요가격대비 편안하고 좋았습니당,2017.10.18,"20,000",대전광역시 유성구 장대로 20-19,유성구,35,50원 할인돼서 되게 싸게 갔어요 가격 대비 편안하고 좋았습니다,36.357106,127.337561
233555,대전 유성 발렌타인,3,잘 머물다왔습니다,2017.10.18,"20,000",대전광역시 유성구 장대로 20-19,유성구,9,잘 머물다 왔습니다,36.357106,127.337561


In [ ]:
# 비정상 데이터 제거
data = data[data['clean_review'].apply(lambda x: isinstance(x, str) and x.strip() != "")]

# 비정상적인 데이터 확인
problematic_rows = data[~data['clean_review'].apply(lambda x: isinstance(x, str) and x.strip() != "")]

print(f"문제 있는 데이터 개수: {len(problematic_rows)}")
print(problematic_rows)

문제 있는 데이터 개수: 0
Empty DataFrame
Columns: [hotel, star, review, date, price, address, district, length, clean_review, latitude, longitude]
Index: []


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
model_name = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

# 라벨링 (3 이상은 긍정, 2 이하는 부정)
data['label'] = data['star'].apply(lambda x: 1 if x >= 3 else 0)

# Hugging Face datasets 형식으로 변환
dataset = Dataset.from_pandas(data[['clean_review', 'label']])

# 데이터 전처리: 토크나이징
def tokenize_function(examples):
    return tokenizer(examples['clean_review'], padding="max_length", truncation=True)

dataset = dataset.map(tokenize_function, batched=True)

# 20000개 샘플로 데이터셋 제한
sampled_dataset = dataset.shuffle(seed=41).select(range(20000))

# 데이터셋 분할 (훈련 80%, 검증 20%)
train_size = int(0.8 * len(sampled_dataset))
train_dataset = sampled_dataset.select(range(train_size))
eval_dataset = sampled_dataset.select(range(train_size, len(sampled_dataset)))

print(f"Train Dataset: {len(train_dataset)} samples")
print(f"Evaluation Dataset: {len(eval_dataset)} samples")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/514 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-7-51620ab75690>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'] = data['star'].apply(lambda x: 1 if x >= 3 else 0)


Map:   0%|          | 0/233535 [00:00<?, ? examples/s]

Train Dataset: 16000 samples
Evaluation Dataset: 4000 samples


In [ ]:
# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./result",                  # 학습 결과 저장 경로
    evaluation_strategy="epoch",            # 매 에포크마다 평가
    learning_rate=5e-5,                     # 학습률 (config 기준)
    per_device_train_batch_size=16,         # 학습 배치 크기 (config 기준)
    per_device_eval_batch_size=16,         # 평가 배치 크기 (config 기준)
    num_train_epochs=3,                    # 학습 에포크 수 (config 기준)
    weight_decay=0.0,                       # 가중치 감쇠 (config 기준)
    logging_dir="./logs",                   # 로그 저장 경로
    logging_steps=2000,                     # 로그 출력 주기 (config 기준)
    save_strategy="steps",                  # 체크포인트 저장 주기
    save_steps=2000,                        # 체크포인트 저장 주기 (config 기준)
    seed=41,                                # 랜덤 시드 값 (config 기준)
    fp16=False,                             # 반정밀도 연산 비활성화 (config에서 `no_cuda=false`로 해석 가능)
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Trainer 설정
trainer = Trainer(
    model=model,                         # 학습시킬 모델
    args=training_args,                  # 학습 설정
    train_dataset=train_dataset,         # 훈련 데이터셋
    eval_dataset=eval_dataset,           # 검증 데이터셋
    tokenizer=tokenizer                  # 토크나이저
)

<ipython-input-15-a3132b4572ab>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# 모델 훈련
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.162525
2,0.137200,0.146999
3,0.137200,0.150645


TrainOutput(global_step=3000, training_loss=0.13541802978515624, metrics={'train_runtime': 4517.1407, 'train_samples_per_second': 10.626, 'train_steps_per_second': 0.664, 'total_flos': 1.262933065728e+16, 'train_loss': 0.13541802978515624, 'epoch': 3.0})

In [ ]:
# 모델 검증
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.15064454078674316, 'eval_runtime': 121.1502, 'eval_samples_per_second': 33.017, 'eval_steps_per_second': 2.064, 'epoch': 3.0}


In [ ]:
# 모델 저장
model.save_pretrained("./finetuned_model4")
tokenizer.save_pretrained("./finetuned_model4")

('./finetuned_model4/tokenizer_config.json',
 './finetuned_model4/special_tokens_map.json',
 './finetuned_model4/tokenizer.json')